In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
import requests

In [ ]:
import pandas as pd

In [ ]:
# GET - запрос
url = 'https://www.kinonews.ru/top250imdb_p2/' # страница со всеми статьями 
page = requests.get(url)

In [ ]:
page.status_code

In [ ]:
soup = bs(page.text, 'html.parser')

In [ ]:
page.text

In [ ]:
soup

In [ ]:
result_list = {'title_russia': [], 'title_english':[], 'date': [], 'country':[], 'genre': [],'duration': [],'rating': [], 'description': []}

In [ ]:
pagenum = 1
for i in range(5):
    url = f'https://www.kinonews.ru/top250imdb_p{pagenum}/'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    
    titles = soup.find_all('a', class_='titlefilm')
    
    for i in titles: 
        # Переход на страницу статьи
        url_film = 'https://www.kinonews.ru' + i.get('href') 
        page_film = requests.get(url_film)
        soup_film = bs(page_film.text, 'html.parser')
        
        # Название фильма
        name_film_rus = soup_film.find('h1', class_='film') 
        name_film_eng = soup_film.find('div', class_='entitle')
        
        # Дата выпуска фильма
        release_date = soup_film.find('td', string="Год выпуска:")
        if release_date:
            release_date = release_date.find_next_sibling('td').text.strip()
        
        # Страны выпуска фильма
        meta_tags = soup_film.find_all('meta', {'itemprop': 'countryOfOrigin'})
        if meta_tags:
            countries = [tag['content'] for tag in meta_tags]
            country = ', '.join(countries)
        else:
            country = 'N/A'  
        # Жанр
        genre_tags = soup_film.find_all('span', itemprop="genre")
        # Берем только первый жанр (если есть)
        recommend_genre = genre_tags[0].text.strip() if genre_tags else 'N/A'
            
        # Длительность фильма        
        duration_film = soup_film.find('td', string="Продолжительность:")
        duration = duration_film.find_next_sibling('td').text.strip() if duration_film else 'N/A'
        
        # Рейтинг
        rating_film = soup_film.find('meta', itemprop="ratingValue")
        rating = rating_film.get('content')
        
        # Описание фильма
        discription_film = soup_film.find('div', itemprop="description") 
        if discription_film:
            discription_film = discription_film.text.strip()
            
            
        if name_film_rus:
            result_list['title_russia'].append(name_film_rus.text.strip())
            result_list['title_english'].append(name_film_eng.text.strip() if name_film_eng else 'N/A')
            result_list['date'].append(release_date)
            result_list['country'].append(country)
            result_list['genre'].append(recommend_genre)
            result_list['duration'].append(duration)
            result_list['rating'].append(rating)
            result_list['description'].append(discription_film)
            
    pagenum += 1

In [ ]:
result_list

In [ ]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

In [ ]:
file_name = 'film.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
pip install gensim

In [ ]:
pip install pymorphy3

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

import re
import nltk
import pymorphy3

from nltk.stem.snowball import SnowballStemmer

In [ ]:
df = pd.read_csv('film.csv')

In [ ]:
df.head()

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.head(7)

In [ ]:
df.info()

In [ ]:
df['description'][0]

In [ ]:
st = '\xa0'
def remove_othersymbol(text):
    return ''.join([ch if ch not in st else ' ' for ch in text])

In [ ]:
string.punctuation

In [ ]:
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in string.punctuation])

In [ ]:
df['prep_text']= [remove_othersymbol(text.lower()) for text in df['description']]

In [ ]:
df['prep_text'][0]

In [ ]:
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in string.punctuation])

In [ ]:
df['prep_text']= [remove_punctuation(text) for text in df['prep_text']]

In [ ]:
df['prep_text'][0]

In [ ]:
df.head()

In [ ]:
russian_stopwords = stopwords.words("russian") 
print(russian_stopwords)

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [ ]:
%%time
lemm_texts_list = []
for text in (df['prep_text']):
    try:
        tokens = word_tokenize(text)
        res = list()
        for word in tokens:
            p = morph.parse(word)[0]
            res.append(p.normal_form)
        text = " ".join(res)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df['text_lemm'] = lemm_texts_list

In [ ]:
df.head()

In [ ]:
def tokenize(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    text = " ".join(tokens)
    return text

In [ ]:
df['tokenize_text'] = [tokenize(text) for text in df['text_lemm']]

In [ ]:
df.head()

In [ ]:
pip install WordCloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def str_corpus(corpus):
    str_corpus = ''
    for i in corpus:
        str_corpus += ' ' + i
    str_corpus = str_corpus.strip()
    return str_corpus

def get_corpus(data):
    corpus = []
    for phrase in data:
        for word in phrase.split():
            corpus.append(word)
    return corpus

def get_wordCloud(corpus):
    wordCloud = WordCloud(background_color='white',
                              stopwords=STOPWORDS,
                              width=3000,
                              height=2500,
                              max_words=200,
                              random_state=42
                         ).generate(str_corpus(corpus))
    return wordCloud

In [ ]:
corpus = get_corpus(df['tokenize_text'].values)
procWordCloud = get_wordCloud(corpus)

fig = plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.imshow(procWordCloud)
plt.axis('off')
plt.subplot(1, 2, 1)
plt.show()

In [ ]:
def get_wordCloud2(corpus):
    wordCloud = WordCloud(background_color='white',
                              stopwords=STOPWORDS,
                              width=3000,
                              height=2500,
                              max_words=200,
                              random_state=42
                         ).generate(corpus)
    return wordCloud

In [ ]:
corpus = df['tokenize_text'][20]
procWordCloud = get_wordCloud2(corpus)

fig = plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.imshow(procWordCloud)
plt.axis('off')
plt.subplot(1, 2, 1)
plt.show()

In [ ]:
corpus = df['tokenize_text'][30]
procWordCloud = get_wordCloud2(corpus)

fig = plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.imshow(procWordCloud)
plt.axis('off')
plt.subplot(1, 2, 1)
plt.show()

In [ ]:
corpus = df['tokenize_text'][40]
procWordCloud = get_wordCloud2(corpus)

fig = plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.imshow(procWordCloud)
plt.axis('off')
plt.subplot(1, 2, 1)
plt.show()

In [ ]:
vectorizer = CountVectorizer()
count_matrix1 = vectorizer.fit(df['tokenize_text'])

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
print(vectorizer.vocabulary_)

In [ ]:
russian_stopwords = list(russian_stopwords)

In [ ]:
vectorizer = CountVectorizer(analyzer='word', stop_words=russian_stopwords, ngram_range=(1, 3), min_df=2)
count_matrix = vectorizer.fit_transform(df['tokenize_text'])

In [ ]:
count_matrix.shape

In [ ]:
print(count_matrix)

In [ ]:
vectorizer.get_feature_names_out()[100:200]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))

In [ ]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(df['tokenize_text'])

In [ ]:
tfidf_matrix.shape

In [ ]:
print(tfidf_matrix)

In [ ]:
tfidf_vectorizer.get_feature_names_out()[150:160]

In [ ]:
import gensim

In [ ]:
def tokenize2(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    return tokens

In [ ]:
df['tokenize_text2'] = [tokenize2(text) for text in df['tokenize_text']]

In [ ]:
df.head()

In [ ]:
gensim_dictionary = gensim.corpora.Dictionary(df['tokenize_text2'])
gensim_dictionary.filter_extremes(no_above=0.1, no_below=5)
gensim_dictionary.compactify()

In [ ]:
corpus = [gensim_dictionary.doc2bow(text) 
          for text in df['tokenize_text2']]

In [ ]:
corpus

In [ ]:
lda_20 = gensim.models.LdaMulticore(corpus, 
                                 num_topics=10, 
                                 id2word=gensim_dictionary, 
                                 passes=10, random_state=6457)

In [ ]:
lda_20.print_topics()

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_20, corpus, gensim_dictionary)
pyLDAvis.display(vis_data)

In [ ]:
pip install pyLDAvis

In [ ]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_20,
                                     texts=df["tokenize_text2"],
                                     dictionary=gensim_dictionary,
                                     coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()

print("\nCoherence Score: ", coherence_lda)

In [ ]:
coherence_values = []
for num_topics in range(10, 30, 5):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                             num_topics=num_topics,
                             id2word=gensim_dictionary,
                             passes=10,
                             random_state=6457)
    
    coherence_model = CoherenceModel(model=lda_model,
                                     texts=df["tokenize_text2"],
                                     dictionary=gensim_dictionary,
                                     coherence="c_v")
    
    coherence_values.append(coherence_model.get_coherence())

import matplotlib.pyplot as plt

plt.plot(range(10, 30, 5), coherence_values, marker='o')
plt.xlabel("Количество тем")
plt.ylabel("Когерентность")
plt.title("Зависимость когерентности от количества тем")
plt.show()

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
vis_20 = gensimvis.prepare(lda_20, corpus, gensim_dictionary)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis_20

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda_model = LatentDirichletAllocation(n_components=10, random_state=0)
lda_model.fit(count_matrix)

for i, topic in enumerate(lda_model.components_):
    print(f"Topic {i}: {', '.join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

In [ ]:
lda_model = LatentDirichletAllocation(learning_method='online', random_state=5, n_jobs=-1, n_components=11)
lda_model.fit(count_matrix)

In [ ]:
topic_keywords = pd.DataFrame()
for i,topic in enumerate(lda_model.components_):
    topic_keywords[f'Topic {i + 1}'] = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-50:]]
topic_keywords

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
nmf_model = NMF(n_components=10, random_state=0)
nmf_model.fit(tfidf_matrix)
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

In [ ]:
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, random_state=0)
lsa_model.fit(tfidf_matrix)

for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

In [ ]:
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

In [ ]:
df.head(25)

In [ ]:
df.info()

In [ ]:
df['duration'].value_counts()

In [ ]:
df['duration'] = df['duration'].str.replace(' мин.', '', regex=False)
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
mean_duration = df['duration'].mean()
df['duration'] = df['duration'].fillna(mean_duration)
print(df['duration'].value_counts(dropna=False))

In [ ]:
df['date'].value_counts()

In [ ]:
df['date'] = pd.to_numeric(df['date'], errors='coerce')
df['date'].value_counts()

In [ ]:
df['rating'].value_counts()

In [ ]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
mean_rating = df['rating'].mean()
df['rating'] = df['rating'].fillna(mean_rating)
df['rating'].value_counts()

In [ ]:
df['genre'].value_counts()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['genre_number'] = label_encoder.fit_transform(df['genre'])

In [ ]:
df['genre_number'].value_counts()

In [ ]:
df['rating'] = df['rating'].astype(float)
df['duration'] = df['duration'].astype(float)
df['date'] = df['date'].astype(float)

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X = df.loc[:, ['date', 'duration', 'rating', 'genre_number']].values

In [ ]:
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(X)
data_normalized = pd.DataFrame(data_normalized, columns=['date', 'duration', 'rating', 'genre_number'])

print("\nДанные после работы нормализации:")
print(data_normalized)

In [ ]:
print(df[['date', 'duration', 'rating', 'genre_number']].isnull().sum())
print(df[['date', 'duration', 'rating', 'genre_number']].dtypes)

In [ ]:
X = tfidf_matrix

In [ ]:
from sklearn.cluster import KMeans

wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 10), wcss)
plt.title('Выбор оптимального значения k')
plt.xlabel('Количество кластеров')
plt.ylabel('WCSS')
plt.show()

In [ ]:
num_clusters = 7
km = KMeans(n_clusters=num_clusters)

In [ ]:
%%time
km.fit(X)

In [ ]:
pred=kmeans.fit_predict(X)
clusters = km.labels_.tolist()

In [ ]:
clusters[:10]

In [ ]:
cluster_kmeans = km.labels_.tolist()
df['cluster'] = cluster_kmeans
df['cluster'].value_counts()

In [ ]:
df[df['cluster']==0]

In [ ]:
df[df['cluster']==1]

In [ ]:
df[df['cluster']==2]

In [ ]:
df[df['cluster']==3]

In [ ]:
df[df['cluster']==4]

In [ ]:
df[df['cluster']==5]

In [ ]:
df[df['cluster']==6]

In [ ]:
top_rated_movies = (df[df['cluster'] == 0]
                   .sort_values('rating', ascending=False)
                   .head(5)[['title_russia', 'rating']])
print("\nТоп-5 фильмов с самым высоким рейтингом в кластере 0:")
print(top_rated_movies)
print("\n")
cluster_0_count = len(df[df['cluster'] == 0])
print(f"Количество фильмов в кластере 0: {cluster_0_count}")

In [ ]:
top_rated_movies = (df[df['cluster'] == 1]
                   .sort_values('rating', ascending=False)
                   .head(5)[['title_russia', 'rating']])
print("\nТоп-5 фильмов с самым высоким рейтингом в кластере 1:")
print(top_rated_movies)
print("\n")
cluster_1_count = len(df[df['cluster'] == 1])
print(f"Количество фильмов в кластере 1: {cluster_1_count}")

In [ ]:
top_rated_movies = (df[df['cluster'] == 2]
                   .sort_values('rating', ascending=False)
                   .head(5)[['title_russia', 'rating']])
print("\nТоп-5 фильмов с самым высоким рейтингом в кластере 2:")
print(top_rated_movies)
print("\n")
cluster_2_count = len(df[df['cluster'] == 2])
print(f"Количество фильмов в кластере 2: {cluster_2_count}")

In [ ]:
top_rated_movies = (df[df['cluster'] == 3]
                   .sort_values('rating', ascending=False)
                   .head(5)[['title_russia', 'rating']])
print("\nТоп-5 фильмов с самым высоким рейтингом в кластере 3:")
print(top_rated_movies)
print("\n")
cluster_3_count = len(df[df['cluster'] == 3])
print(f"Количество фильмов в кластере 3: {cluster_3_count}")

In [ ]:
top_rated_movies = (df[df['cluster'] == 4]
                   .sort_values('rating', ascending=False)
                   .head(5)[['title_russia', 'rating']])
print("\nТоп-5 фильмов с самым высоким рейтингом в кластере 4:")
print(top_rated_movies)
print("\n")
cluster_4_count = len(df[df['cluster'] == 4])
print(f"Количество фильмов в кластере 4: {cluster_4_count}")

In [ ]:
top_rated_movies = (df[df['cluster'] == 5]
                   .sort_values('rating', ascending=False)
                   .head(5)[['title_russia', 'rating']])
print("\nТоп-5 фильмов с самым высоким рейтингом в кластере 5:")
print(top_rated_movies)
print("\n")
cluster_5_count = len(df[df['cluster'] == 5])
print(f"Количество фильмов в кластере 5: {cluster_5_count}")

In [ ]:
top_rated_movies = (df[df['cluster'] == 6]
                   .sort_values('rating', ascending=False)
                   .head(5)[['title_russia', 'rating']])
print("\nТоп-5 фильмов с самым высоким рейтингом в кластере 6:")
print(top_rated_movies)
print("\n")
cluster_6_count = len(df[df['cluster'] == 6])
print(f"Количество фильмов в кластере 6: {cluster_6_count}")

In [ ]:
df['cluster'] = cluster_kmeans
cluster_stats = df.groupby('cluster').agg({
    'duration': ['mean', 'median', 'min', 'max'],
    'date': ['mean', 'median', 'min', 'max'],
    'rating': ['mean', 'median', 'min', 'max', 'count']
}).round(2)

print(cluster_stats)

In [ ]:
def tokenize3(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    return tokens
df.head()


In [ ]:
n_clusters = 7
n_top_words = 10

for cluster_id in range(n_clusters):
    cluster_texts = df[df['cluster'] == cluster_id]['title_russia']
    
    vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words=russian_stopwords)
    X = vectorizer.fit_transform(cluster_texts)
    
    lda = LatentDirichletAllocation(n_components=4, random_state=42)  
    lda.fit(X)
    
    feature_names = vectorizer.get_feature_names_out()
    print(f"\nКластер {cluster_id}:")
    for topic_idx, topic in enumerate(lda.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print(f"Topic {topic_idx}: {', '.join(top_words)}")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['tokenize_text'], df['cluster'], 
                                                      test_size=0.3, 
                                                      random_state=0)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,3), stop_words=russian_stopwords)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = model_rf.predict(X_test_tfidf)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = model_knn.predict(X_test_tfidf)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = model_lr.predict(X_test_tfidf)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
decision_tree_regressor = DecisionTreeClassifier()
decision_tree_regressor.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = decision_tree_regressor.predict(X_test_tfidf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
y_pred

In [ ]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model_knn, f)

In [ ]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [ ]:
df.to_csv('film.csv')